In [184]:
import pandas as pd
import numpy as np
data = pd.read_csv('three_daily_price.csv')

In [185]:
data.head(10)

,PERMNO,date,TICKER,COMNAM,PRC
0,10001,19890703,GFGC,GREAT FALLS GAS CO,7.125
1,10001,19890705,GFGC,GREAT FALLS GAS CO,-7.125
2,10001,19890706,GFGC,GREAT FALLS GAS CO,7.000
3,10001,19890707,GFGC,GREAT FALLS GAS CO,7.250
4,10001,19890710,GFGC,GREAT FALLS GAS CO,7.125
5,10001,19890711,GFGC,GREAT FALLS GAS CO,7.125
6,10001,19890712,GFGC,GREAT FALLS GAS CO,7.000
7,10001,19890713,GFGC,GREAT FALLS GAS CO,7.000
8,10001,19890714,GFGC,GREAT FALLS GAS CO,7.500
9,10001,19890717,GFGC,GREAT FALLS GAS CO,-7.250


In [186]:
data.drop(['TICKER','COMNAM'],axis=1,inplace = True)

In [187]:
data['year']=data['date'].apply(str)
data['year']=data['year'].str[:4]

In [188]:
import datetime
def date(a):
    date =str(a)
    datetime_object = datetime.datetime(int(date[:4]),int(date[4:6]),int(date[6:]))
    return datetime_object

In [189]:
data['Date'] = data['date'].apply(date)

In [190]:
data['weekday'] = data['Date'].apply(lambda x: x.weekday()+1)

In [191]:
data['month'] = data['Date'].apply(lambda x: x.month)

In [192]:
data.head()

,PERMNO,date,PRC,year,Date,weekday,month
0,10001,19890703,7.125,1989,1989-07-03,1,7
1,10001,19890705,-7.125,1989,1989-07-05,3,7
2,10001,19890706,7.000,1989,1989-07-06,4,7
3,10001,19890707,7.250,1989,1989-07-07,5,7
4,10001,19890710,7.125,1989,1989-07-10,1,7


#### Relative Strength Index

In [193]:
def UC_DC(data):
    data['UC'] = data['PRC'] - data['PRC'].shift(1)
    data['DC'] = data['PRC'] - data['PRC'].shift(1)
    return data
    

In [194]:
data=data.groupby(['PERMNO']).apply(UC_DC)              

In [195]:
data['UC'] = data['UC'].apply(lambda x: 0 if x<0 else x)
data['DC'] = data['DC'].apply(lambda x: 0 if x>0 else -x)

In [196]:
data_weekly = data[data['weekday'] == 5]

In [197]:
data_weekly.reset_index(inplace = True)
data_weekly.drop(['index'],axis = 1,inplace = True)

/Users/jingsixu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [198]:
def AUC_ADC(data_weekly):
    data_weekly['AUC']= data_weekly['UC'].rolling(43).mean()
    data_weekly['ADC']= data_weekly['DC'].rolling(43).mean()
    #return data_weekly


    data_weekly.reset_index(inplace=True)
    for i in range (43,data_weekly.shape[0]):
        data_weekly.loc[i,'AUC']= (data_weekly.loc[i-1,'AUC']*42+ data_weekly.loc[i,'UC'])/43
    for i in range (43,data_weekly.shape[0]):
        data_weekly.loc[i,'ADC']= (data_weekly.loc[i-1,'ADC']*42+ data_weekly.loc[i,'DC'])/43
    return data_weekly

In [199]:
data_weekly=data_weekly.groupby('PERMNO',as_index=False).apply(AUC_ADC).reset_index()

In [200]:
data_weekly.drop(['level_0','level_1','index'],axis=1,inplace = True)

In [201]:
data_weekly['RSI'] = 100 - (100/(1+ data_weekly['AUC']/data_weekly['ADC']))

#### MACD

In [202]:
def EMA1226(data_weekly):
    data_weekly['EMA_12']=data_weekly['PRC'].rolling(12).mean()
    data_weekly['EMA_26']=data_weekly['PRC'].rolling(26).mean()
    
    k_12=2.0/(12.0+1.0)
    k_26=2.0/(26.0+1.0)
    
    data_weekly.reset_index(inplace=True)
    for i in range (12,data_weekly.shape[0]):
        data_weekly.loc[i,'EMA_12']=data_weekly.loc[i-1,'EMA_12']*(1-k_12)+ data_weekly.loc[i,'PRC']*k_12
        
    for i in range (26,data_weekly.shape[0]):
        data_weekly.loc[i,'EMA_26']=data_weekly.loc[i-1,'EMA_26']*(1-k_26)+ data_weekly.loc[i,'PRC']*k_26
    return data_weekly

In [203]:
data_weekly=data_weekly.groupby('PERMNO',as_index=False).apply(EMA1226).reset_index()

In [204]:
data_weekly.drop(['level_0','level_1','index'],axis=1,inplace = True)

In [205]:
data_weekly['MACD'] = data_weekly['EMA_12'] - data_weekly['EMA_26']

In [206]:
def signal_line(data_weekly):
    data_weekly['Signal_Line']=data_weekly['MACD'].rolling(9).mean()
    
    k_9=2.0/(9.0+1.0)
    data_weekly.reset_index(inplace=True)
    for i in range (34,data_weekly.shape[0]):
        data_weekly.loc[i,'Signal_Line']=data_weekly.loc[i-1,'Signal_Line']*(1-k_9)+ data_weekly.loc[i,'MACD']*k_9
    
    return data_weekly

In [207]:
data_weekly = data_weekly.groupby('PERMNO',as_index=False).apply(signal_line).reset_index()

In [208]:
data_weekly['MACD-Signal_Line']=data_weekly['MACD']-data_weekly['Signal_Line']

In [209]:
data_weekly.drop(['level_0','level_1','index'],axis=1,inplace = True)

In [211]:
data_weekly.head(60)

,PERMNO,date,PRC,year,Date,weekday,month,UC,DC,AUC,ADC,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD-Signal_Line
0,10001,19890707,7.2500,1989,1989-07-07,5,7,0.2500,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,19890714,7.5000,1989,1989-07-14,5,7,0.5000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,19890721,7.5000,1989,1989-07-21,5,7,0.5000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,19890728,7.5000,1989,1989-07-28,5,7,0.2500,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,19890804,7.7500,1989,1989-08-04,5,8,0.0000,-0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10001,19890811,7.5000,1989,1989-08-11,5,8,0.0000,0.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10001,19890818,7.5000,1989,1989-08-18,5,8,0.0000,0.1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10001,19890825,7.7500,1989,1989-08-25,5,8,0.0000,-0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10001,19890901,9.0000,1989,1989-09-01,5,9,0.0000,0.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10001,19890908,9.2500,1989,1989-09-08,5,9,0.0000,-0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### BBP

In [177]:
def bbp(data_weekly):
    data_weekly['SMA']=data_weekly['PRC'].rolling(20).mean()
    data_weekly['STD']=data_weekly['PRC'].rolling(20).mean()
    #data_weekly3['Top_Bound']=data_weekly3['SMA']+2*data_weekly3['STD']
    #data_weekly3['Bottom_Bound']=data_weekly3['SMA']-2*data_weekly3['STD']
    data_weekly['BBP']=(data_weekly['PRC']-data_weekly['SMA'])/(2*data_weekly['STD'])
    return data_weekly

In [178]:
data_weekly=data_weekly.groupby('PERMNO',as_index=False).apply(bbp)

#### post-work

In [179]:
data_weekly = data_weekly[data_weekly['month']==6]

In [180]:
RSI_MACD_BBP = data_weekly.groupby(['PERMNO','year']).tail(1)

In [181]:
RSI_MACD_BBP = RSI_MACD_BBP[['PERMNO','Date','year','RSI','MACD-Signal_Line','BBP']]

In [182]:
RSI_MACD_BBP.head(50)

,PERMNO,Date,year,RSI,MACD-Signal_Line,BBP
50,10001,1990-06-29,1990,61.458555,0.209721,0.203654
101,10001,1991-06-28,1991,55.258518,0.171346,0.549344
152,10001,1992-06-26,1992,62.205760,0.210034,0.153168
200,10001,1993-06-25,1993,57.113107,1.853461,1.900000
250,10001,1994-06-24,1994,52.690614,-2.209677,-3.533708
302,10001,1995-06-30,1995,64.802851,-0.393748,0.849003
353,10001,1996-06-28,1996,44.769440,0.500303,0.933772
404,10001,1997-06-27,1997,53.097258,-0.422527,0.057782
454,10001,1998-06-26,1998,41.941388,0.119974,0.340734
502,10001,1999-06-25,1999,38.608018,-0.324241,0.305604


In [183]:
RSI_MACD_BBP.to_csv('rsi_macd_bbp.csv')